In [1]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime

1.8.1
0.8.0a0+e4e171a


In [24]:
threshold = 0.5
checkpoint = 'resnest50-full-training-20210503-174656-latest.pt'

In [25]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [26]:
labels = ['acafly', 'acowoo', 'aldfly', 'ameavo', 'amecro', 'amegfi', 'amekes', 'amepip', 'amered', 'amerob', 'amewig', 'amtspa', 'andsol1',                                                                                                                                                       
 'annhum', 'astfly', 'azaspi1', 'babwar', 'baleag', 'balori', 'banana', 'banswa', 'banwre1', 'barant1', 'barswa', 'batpig1', 'bawswa1',                                                                                                                                                   
 'bawwar', 'baywre1', 'bbwduc', 'bcnher', 'belkin1', 'belvir', 'bewwre', 'bkbmag1', 'bkbplo', 'bkbwar', 'bkcchi', 'bkhgro', 'bkmtou1',                                                                                                                                                    
 'bknsti', 'blbgra1', 'blbthr1', 'blcjay1', 'blctan1', 'blhpar1', 'blkpho', 'blsspa1', 'blugrb1', 'blujay', 'bncfly', 'bnhcow',                                                                                                                                                           
 'bobfly1', 'bongul', 'botgra', 'brbmot1', 'brbsol1', 'brcvir1', 'brebla', 'brncre', 'brnjay', 'brnthr', 'brratt1', 'brwhaw', 'brwpar1',                                                                                                                                                  
 'btbwar', 'btnwar', 'btywar', 'bucmot2', 'buggna', 'bugtan', 'buhvir', 'bulori', 'burwar1', 'bushti', 'butsal1', 'buwtea', 'cacgoo1',                                                                                                                                                    
 'cacwre', 'calqua', 'caltow', 'cangoo', 'canwar', 'carchi', 'carwre', 'casfin', 'caskin', 'caster1', 'casvir', 'categr', 'ccbfin',                                                                                                                                                       
 'cedwax', 'chbant1', 'chbchi', 'chbwre1', 'chcant2', 'chispa', 'chswar', 'cinfly2', 'clanut', 'clcrob', 'cliswa', 'cobtan1', 'cocwoo1',                                                                                                                                                  
 'cogdov', 'colcha1', 'coltro1', 'comgol', 'comgra', 'comloo', 'commer', 'compau', 'compot1', 'comrav', 'comyel', 'coohaw', 'cotfly1',                                                                                                                                                    
 'cowscj1', 'cregua1', 'creoro1', 'crfpar', 'cubthr', 'daejun', 'dowwoo', 'ducfly', 'dusfly', 'easblu', 'easkin', 'easmea', 'easpho',                                                                                                                                                     
 'eastow', 'eawpew', 'eletro', 'eucdov', 'eursta', 'fepowl', 'fiespa', 'flrtan1', 'foxspa', 'gadwal', 'gamqua', 'gartro1', 'gbbgul',                                                                                                                                                      
 'gbwwre1', 'gcrwar', 'gilwoo', 'gnttow', 'gnwtea', 'gocfly1', 'gockin', 'gocspa', 'goftyr1', 'gohque1', 'goowoo1', 'grasal1',                                                                                                                                                            
 'grbani', 'grbher3', 'grcfly', 'greegr', 'grekis', 'grepew', 'grethr1', 'gretin1', 'greyel', 'grhcha1', 'grhowl', 'grnher', 'grnjay',                                                                                                                                                    
 'grtgra', 'grycat', 'gryhaw2', 'gwfgoo', 'haiwoo', 'heptan', 'hergul', 'herthr', 'herwar', 'higmot1', 'hofwoo1', 'houfin', 'houspa',                                                                                                                                                     
 'houwre', 'hutvir', 'incdov', 'indbun', 'kebtou1', 'killde', 'labwoo', 'larspa', 'laufal1', 'laugul', 'lazbun', 'leafly', 'leasan',                                                                                                                                                      
 'lesgol', 'lesgre1', 'lesvio1', 'linspa', 'linwoo1', 'littin1', 'lobdow', 'lobgna5', 'logshr', 'lotduc', 'lotman1', 'lucwar', 'macwar',                                                                                                                                                  
 'magwar', 'mallar3', 'marwre', 'mastro1', 'meapar', 'melbla1', 'monoro1', 'mouchi', 'moudov', 'mouela1', 'mouqua', 'mouwar', 'mutswa',                                                                                                                                                   
 'naswar', 'norcar', 'norfli', 'normoc', 'norpar', 'norsho', 'norwat', 'nrwswa', 'nutwoo', 'oaktit', 'obnthr1', 'ocbfly1', 'oliwoo1',                                                                                                                                                     
 'olsfly', 'orbeup1', 'orbspa1', 'orcpar', 'orcwar', 'orfpar', 'osprey', 'ovenbi1', 'pabspi1', 'paltan1', 'palwar', 'pasfly', 'pavpig2',                                                                                                                                                  
 'phivir', 'pibgre', 'pilwoo', 'pinsis', 'pirfly1', 'plawre1', 'plaxen1', 'plsvir', 'plupig2', 'prowar', 'purfin', 'purgal2', 'putfru1',                                                                                                                                                  
 'pygnut', 'rawwre1', 'rcatan1', 'rebnut', 'rebsap', 'rebwoo', 'redcro', 'reevir1', 'rehbar1', 'relpar', 'reshaw', 'rethaw', 'rewbla',                                                                                                                                                    
 'ribgul', 'rinkin1', 'roahaw', 'robgro', 'rocpig', 'rotbec', 'royter1', 'rthhum', 'rtlhum', 'ruboro1', 'rubpep1', 'rubrob', 'rubwre1',                                                                                                                                                   
 'ruckin', 'rucspa1', 'rucwar', 'rucwar1', 'rudpig', 'rudtur', 'rufhum', 'rugdov', 'rumfly1', 'runwre1', 'rutjac1', 'saffin', 'sancra',                                                                                                                                                   
 'sander', 'savspa', 'saypho', 'scamac1', 'scatan', 'scbwre1', 'scptyr1', 'scrtan1', 'semplo', 'shicow', 'sibtan2', 'sinwre1', 'sltred',                                                                                                                                                  
 'smbani', 'snogoo', 'sobtyr1', 'socfly1', 'solsan', 'sonspa', 'soulap1', 'sposan', 'spotow', 'spvear1', 'squcuc1', 'stbori', 'stejay',                                                                                                                                                   
 'sthant1', 'sthwoo1', 'strcuc1', 'strfly1', 'strsal1', 'stvhum2', 'subfly', 'sumtan', 'swaspa', 'swathr', 'tenwar', 'thbeup1', 'thbkin',                                                                                                                                                 
 'thswar1', 'towsol', 'treswa', 'trogna1', 'trokin', 'tromoc', 'tropar', 'tropew1', 'tuftit', 'tunswa', 'veery', 'verdin', 'vigswa',                                                                                                                                                      
 'warvir', 'wbwwre1', 'webwoo1', 'wegspa1', 'wesant1', 'wesblu', 'weskin', 'wesmea', 'westan', 'wewpew', 'whbman1', 'whbnut', 'whcpar',                                                                                                                                                   
 'whcsee1', 'whcspa', 'whevir', 'whfpar1', 'whimbr', 'whiwre1', 'whtdov', 'whtspa', 'whwbec1', 'whwdov', 'wilfly', 'willet1', 'wilsni1',                                                                                                                                                  
 'wiltur', 'wlswar', 'wooduc', 'woothr', 'wrenti', 'y00475', 'yebcha', 'yebela1', 'yebfly', 'yebori1', 'yebsap', 'yebsee1', 'yefgra1',                                                                                                                                                    
 'yegvir', 'yehbla', 'yehcar1', 'yelgro', 'yelwar', 'yeofly1', 'yerwar', 'yeteup1', 'yetvir'] 

len(labels)

397

In [27]:
class MixedSig2Spec(torch.nn.Module):
    def __init__(self):
        super().__init__()

        window_lengths = [800, 1600, 3200]
        hop_lengths = [320, 800, 1600]

        self.melspecs = [tat.MelSpectrogram(
            sample_rate=32000,
            n_fft=window_lengths[i],
            win_length=window_lengths[i],
            hop_length=hop_lengths[i],
            f_min=150,
            f_max=15000,
            pad=0,
            n_mels=128,
            power=2.0,
            normalized=False,
        ).to(device) for i in range(3)]
        self.p2db = tat.AmplitudeToDB(stype='power', top_db=80)
        self.tf_resize = transforms.Resize((128, 250))
        self.tf_norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def forward(self, sig: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        # note: assuming batch input
        with torch.no_grad():
            imgs = [self.tf_resize(self.p2db(ms(sig))) for ms in self.melspecs]  # 3 * [bs x 1 x H x W]
            spec = torch.cat([x.transpose(0, 1) for x in imgs]).transpose(0, 1)
            spec = self.tf_norm(spec)
            return spec


In [28]:
# prep = MixedSig2Spec()
# main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
# posp = torch.nn.Linear(in_features=2048, out_features=len(labels), bias=True)
# main_model.fc = posp
# model = torch.nn.Sequential(prep, main_model).to(device)
# model = model.eval()

In [29]:
prep = MixedSig2Spec()
# main_model = torch.hub.load('../input/zhanghang1989-resnest-master/zhanghang1989_ResNeSt_master', 'resnest50', pretrained=False, source='local')
main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=False)
posp = torch.nn.Linear(in_features=2048, out_features=len(labels), bias=True)
main_model.fc = posp
model = torch.nn.Sequential(prep, main_model).to(device)
cp = torch.load(f"checkpoints/{checkpoint}", map_location=torch.device(device))
model.load_state_dict(cp['model_state_dict'])
model = model.eval()

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [30]:
def list_files(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.rsplit('.', 1)[-1] in ['ogg']]

test_audio = list_files('../input/birdclef-2021/test_soundscapes')
if len(test_audio) == 0:
    test_audio = list_files('../input/birdclef-2021/train_soundscapes')
    
print('{} files in test set.'.format(len(test_audio)))

20 files in test set.


In [31]:
def process_preds(ps, path, threshold=0.5):
    fileinfo = path.split(os.sep)[-1].rsplit('.', 1)[0].split('_')
    prefix = fileinfo[0] + '_'  + fileinfo[1] + '_' 
    result = {'row_id': [], 'birds': []}
    for row, r in enumerate(ps):
        birds = ""
        row_id = prefix + str((row + 1) * 5)  
        for col, c in enumerate(r):
            if c >= threshold:
                b = labels[col]
                birds += f"{b} "
        if birds == "":
            birds = "nocall"
        result['row_id'].append(row_id)
        result['birds'].append(birds)
    return pd.DataFrame(result)

In [32]:
df_result = pd.DataFrame()

with torch.no_grad():
    for path in test_audio:
        print(f"processing {path}")
        sig, sr = ta.load(filepath=path)
        sig = sig.to(device).reshape(-1, 1, 5*32000)
        p = model(sig).cpu().numpy()
        del sig
        df = process_preds(p, path, threshold)
        df_result = df_result.append(df)

processing ../input/birdclef-2021/train_soundscapes/54955_SSW_20170617.ogg
processing ../input/birdclef-2021/train_soundscapes/7019_COR_20190904.ogg
processing ../input/birdclef-2021/train_soundscapes/57610_COR_20190904.ogg
processing ../input/birdclef-2021/train_soundscapes/21767_COR_20190904.ogg
processing ../input/birdclef-2021/train_soundscapes/26746_COR_20191004.ogg
processing ../input/birdclef-2021/train_soundscapes/18003_COR_20190904.ogg
processing ../input/birdclef-2021/train_soundscapes/26709_SSW_20170701.ogg
processing ../input/birdclef-2021/train_soundscapes/11254_COR_20190904.ogg
processing ../input/birdclef-2021/train_soundscapes/2782_SSW_20170701.ogg
processing ../input/birdclef-2021/train_soundscapes/20152_SSW_20170805.ogg
processing ../input/birdclef-2021/train_soundscapes/7954_COR_20190923.ogg
processing ../input/birdclef-2021/train_soundscapes/10534_SSW_20170429.ogg
processing ../input/birdclef-2021/train_soundscapes/31928_COR_20191004.ogg
processing ../input/birdclef

In [33]:
df_result

,row_id,birds
0,54955_SSW_5,nocall
1,54955_SSW_10,nocall
2,54955_SSW_15,nocall
3,54955_SSW_20,nocall
4,54955_SSW_25,nocall
...,...,...
115,14473_SSW_580,nocall
116,14473_SSW_585,nocall
117,14473_SSW_590,nocall
118,14473_SSW_595,nocall


In [34]:
df_result[df_result['birds'] != 'nocall']

,row_id,birds
17,7019_COR_90,bucmot2
21,7019_COR_110,bucmot2
53,57610_COR_270,solsan
100,57610_COR_505,obnthr1
109,57610_COR_550,solsan
...,...,...
8,50878_COR_45,grhcha1
29,50878_COR_150,grhcha1
108,50878_COR_545,solsan
117,50878_COR_590,bobfly1


In [21]:
df_result.to_csv("submission.csv", index=False)